# Prison Allocation problem 
This section will cover the prison allocation task only, we will not in this example include the early release functionality as proof that this model functions as exptected

In [1]:
%pip install -r resources.txt

  Cloning https://github.com/conjure-cp/conjure-notebook.git (to revision v0.0.8) to c:\users\rrhmc\appdata\local\temp\pip-req-build-545e5mp1
  Resolved https://github.com/conjure-cp/conjure-notebook.git to commit 880fd7578cbad5190ffd68e81682f23ff17914ab
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/conjure-cp/conjure-notebook.git 'C:\Users\RRHMc\AppData\Local\Temp\pip-req-build-545e5mp1'
  Running command git checkout -q 880fd7578cbad5190ffd68e81682f23ff17914ab

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\RRHMc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Step 1 
Lets load in the prison template into this session. For context, the prison data contains an example of a prison system with capacity 270 and population 250 (15 empty beds). The 15 empty beds are distributed across different genders, supervision levels, and age categories:

**Adult High Supervision:**
- 2 Male beds: S1W1A-C08-B2, S1W1A-C08-B3
- 1 Female bed: S1W1B-C05-B3

**Adult Medium Supervision:**
- 2 Male beds: S1W2A-C07-B2, S1W2A-C07-B3
- 1 Female bed: S1W2B-C04-B4

**Adult Low Supervision:**
- 2 Male beds: S2W1A-C08-B1, S2W1A-C08-B3
- 1 Female bed: S2W1B-C05-B3

**Young Offenders High Supervision:**
- 1 Female bed: S3W1B-C05-B2

**Young Offenders Low Supervision:**
- 5 Male beds: S3W2A-C07-B3, S3W2A-C07-B4, S3W2C-C07-B4, S3W2C-C10-B4, S3W2C-C11-B4

In [ ]:
import json
import pandas as pd
import numpy as np

# Load the JSON file
with open('Data\PrisonTemplate.json', 'r') as f:
    data = json.load(f)

# Access the prison data
prison = data['prison']

# Flatten the nested structure into a list of bed records
bed_records = []

for section in prison['sections']:
    section_id = section['section_id']
    section_name = section['section_name']
    age_category = section['age_category']
    
    for ward in section['wards']:
        ward_id = ward['ward_id']
        supervision_level = ward['supervision_level']
        
        for wing in ward['wings']:
            wing_id = wing['wing_id']
            sex_assignment = wing['sex_assignment']
            
            for cell in wing['cells']:
                cell_id = cell['cell_id']
                cell_type = cell['cell_type']
                
                for bed in cell['beds']:
                    bed_record = {
                        'section_id': section_id,
                        'section_name': section_name,
                        'age_category': age_category,
                        'ward_id': ward_id,
                        'supervision_level': supervision_level,
                        'wing_id': wing_id,
                        'sex_assignment': sex_assignment,
                        'cell_id': cell_id,
                        'cell_type': cell_type,
                        'bed_id': bed['bed_id'],
                        'occupied': bed['occupied'],
                        'prisoner_id': bed['prisoner_id']
                    }
                    bed_records.append(bed_record)

# Create DataFrame
df_beds = pd.DataFrame(bed_records)

print(f"Total beds in dataset: {len(df_beds)}")
print(f"Occupied beds: {df_beds['occupied'].sum()}")
print(f"Empty beds: {(~df_beds['occupied']).sum()}")
print(f"\nDataFrame shape: {df_beds.shape}")
print(f"\nFirst few rows:")
df_beds.head()

Total beds in dataset: 270
Occupied beds: 255
Empty beds: 15

DataFrame shape: (270, 12)

First few rows:


,section_id,section_name,age_category,ward_id,supervision_level,wing_id,sex_assignment,cell_id,cell_type,bed_id,occupied,prisoner_id
0,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C01,single,S1W1A-C01-B1,True,P00001
1,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C02,single,S1W1A-C02-B1,True,P00002
2,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C03,single,S1W1A-C03-B1,True,P00003
3,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C04,single,S1W1A-C04-B1,True,P00004
4,S1,North Wing Section,Adult,S1W1,High,S1W1A,Male,S1W1A-C05,single,S1W1A-C05-B1,True,P00005


In [3]:
cell_spaces = df_beds[df_beds['occupied'] == 0]
print(f"\nTotal empty beds available for allocation: {len(cell_spaces)}")
print(f"\n List all available beds: ")
print(cell_spaces)


Total empty beds available for allocation: 15

 List all available beds: 
    section_id        section_name     age_category ward_id supervision_level  \
14          S1  North Wing Section            Adult    S1W1              High   
15          S1  North Wing Section            Adult    S1W1              High   
26          S1  North Wing Section            Adult    S1W1              High   
47          S1  North Wing Section            Adult    S1W2            Medium   
48          S1  North Wing Section            Adult    S1W2            Medium   
62          S1  North Wing Section            Adult    S1W2            Medium   
98          S2  South Wing Section            Adult    S2W1               Low   
100         S2  South Wing Section            Adult    S2W1               Low   
120         S2  South Wing Section            Adult    S2W1               Low   
176         S3   East Wing Section  Young Offenders    S3W1              High   
202         S3   East Wing Section

In [4]:
#convert all non numerical data to ordinal data
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
non_numerical_cols = df_beds.select_dtypes(include=['object', 'bool']).columns
df_beds[non_numerical_cols] = encoder.fit_transform(df_beds[non_numerical_cols])    
print(f"\nDataFrame after encoding non-numerical data:")
df_beds.head()

# Recreate cell_spaces after encoding
cell_spaces = df_beds[df_beds['occupied'] == 0].copy()
print(f"\nEmpty beds after encoding: {len(cell_spaces)}")


DataFrame after encoding non-numerical data:

Empty beds after encoding: 15


In [ ]:
#Now lets try and load in the prisoners to be allocated 
with open('Data\AllocatedPrisonerList.json', 'r') as f:
    data = json.load(f)

prisoners = data["incoming_prisoners"]
#Now only get the first 15 prisoners
prisoners = prisoners[:15]

#Display the prisoners to be allocated
print(f"\nTotal prisoners to be allocated: {len(prisoners)}")
print("\nFirst few prisoners:")
for i, prisoner in enumerate(prisoners[:3]):
    print(prisoner)

# Create a mapping from the original encoder
# Get the column names and their encoded values
col_mapping = {}
for i, col in enumerate(non_numerical_cols):
    categories = encoder.categories_[i]
    col_mapping[col] = {cat: idx for idx, cat in enumerate(categories)}

# Now encode the prisoner data using the mapping
prisoners_encoded = []
index= 1
for prisoner in prisoners:
    prisoner_encoded = prisoner.copy()
    prisoner_encoded['prisoner_id']= len(df_beds) + index
    index += 1
    # Map the categorical values using the same encoding
    prisoner_encoded['age_category_encoded'] = col_mapping['age_category'][prisoner['age_category']]
    prisoner_encoded['sex_encoded'] = col_mapping['sex_assignment'][prisoner['sex']]
    prisoner_encoded['supervision_level_encoded'] = col_mapping['supervision_level'][prisoner['supervision_level']]
    # Keep the original categorical columns for later reference
    prisoners_encoded.append(prisoner_encoded)

print(f"\nFirst prisoner after encoding:")
print(prisoners_encoded[0])


Total prisoners to be allocated: 15

First few prisoners:
{'prisoner_id': 'P00266', 'name': 'Angus "Mad Dog" MacDuff', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High'}
{'prisoner_id': 'P00267', 'name': 'Bruce McLean', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High'}
{'prisoner_id': 'P00268', 'name': 'Ailsa Morrison', 'sex': 'Female', 'age_category': 'Adult', 'supervision_level': 'High'}

First prisoner after encoding:
{'prisoner_id': 271, 'name': 'Angus "Mad Dog" MacDuff', 'age_category_encoded': 0, 'sex_encoded': 1, 'supervision_level_encoded': 0}


In [6]:
# Now lets prep our data for the model 
#Incoming prisoners will contain all but the prisoner name
incoming_prisoners_sex= {}
incoming_prisoners_age = {}
incoming_prisoners_supervision = {}

prison_beds_sex= {}
prison_beds_age = {}
prison_beds_supervision = {}

# Use a counter for bed IDs instead of encoded bed_id values
bed_counter = 1

# Store the prisoner IDs for creating the range
prisoner_ids = []

for prisoner in prisoners_encoded:
    prisoner_id = int(prisoner['prisoner_id'])
    prisoner_ids.append(prisoner_id)
    incoming_prisoners_sex[prisoner_id] = int(prisoner['sex_encoded'])
    incoming_prisoners_age[prisoner_id] = int(prisoner['age_category_encoded'])
    incoming_prisoners_supervision[prisoner_id] = int(prisoner['supervision_level_encoded']) 

for index, row in cell_spaces.iterrows():
    # Use sequential bed IDs starting from 1
    prison_beds_sex[bed_counter] = int(row['sex_assignment'])
    prison_beds_age[bed_counter] = int(row['age_category'])                
    prison_beds_supervision[bed_counter] = int(row['supervision_level'])
    bed_counter += 1

# Create ranges for the domains
prisoner_min = min(prisoner_ids)
prisoner_max = max(prisoner_ids)
bed_min = 1
bed_max = bed_counter - 1

print(f"\nPrepared {len(prisoner_ids)} incoming prisoners")
print(f"Prepared {bed_max} available bed spots")

print(f"\nPrisoner ID range: {prisoner_min} to {prisoner_max}")
print(f"Bed ID range: {bed_min} to {bed_max}")
print(f"\nIncoming Prisoners Sex: {incoming_prisoners_sex}")
print(f"\nIncoming Prisoners Age: {incoming_prisoners_age}")
print(f"\nIncoming Prisoners Supervision: {incoming_prisoners_supervision}")
print(f"\nPrison Bed Sex: {prison_beds_sex}")
print(f"\nPrison Bed Age: {prison_beds_age}")
print(f"\nPrison Bed Supervision: {prison_beds_supervision}")


Prepared 15 incoming prisoners
Prepared 15 available bed spots

Incoming Prisoners: {271: 271, 272: 272, 273: 273, 274: 274, 275: 275, 276: 276, 277: 277, 278: 278, 279: 279, 280: 280, 281: 281, 282: 282, 283: 283, 284: 284, 285: 285}

Prison Beds Spots: {14: 14, 15: 15, 25: 25, 46: 46, 47: 47, 61: 61, 97: 97, 99: 99, 119: 119, 175: 175, 201: 201, 202: 202, 252: 252, 264: 264, 268: 268}


In [7]:
%load_ext conjure 

ImportError: cannot import name 'display' from 'IPython.core.display' (C:\Users\RRHMc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\display.py)

In [ ]:
%%conjure
language Essence 1.3

$ Define ranges for prisoners and beds
given prisoner_min, prisoner_max, bed_min, bed_max : int

$ Create domains using the ranges
letting Prisoners be domain int(prisoner_min..prisoner_max),
        Beds be domain int(bed_min..bed_max)

$ Define attribute functions (partial functions, only defined for valid IDs)
given incoming_prisoners_sex, incoming_prisoners_age, incoming_prisoners_supervision: function int --> int
given prison_beds_sex, prison_beds_age, prison_beds_supervision: function int --> int

$ Decision variable: assign each prisoner to a bed (injective ensures one-to-one mapping)
find assignment: function Prisoners --> Beds

such that
$ All prisoners must be assigned (total function)
    forAll prisoner : Prisoners .
        prisoner in defined(assignment),
        
$ Each bed is assigned to at most one prisoner (injective)
    forAll p1, p2 : Prisoners .
        (p1 != p2 /\ p1 in defined(assignment) /\ p2 in defined(assignment)) 
        -> assignment(p1) != assignment(p2),
    
$ Prisoners must be assigned to beds that match their sex
    forAll prisoner : Prisoners .
        prisoner in defined(assignment) ->
        incoming_prisoners_sex(prisoner) = prison_beds_sex(assignment(prisoner)),
    
$ Prisoners must be assigned to beds that match their age category
    forAll prisoner : Prisoners .
        prisoner in defined(assignment) ->
        incoming_prisoners_age(prisoner) = prison_beds_age(assignment(prisoner)),
    
$ Prisoners must be assigned to beds that match their supervision level
    forAll prisoner : Prisoners .
        prisoner in defined(assignment) ->
        incoming_prisoners_supervision(prisoner) = prison_beds_supervision(assignment(prisoner))

In [ ]:
# Iterate through assignments with index
for i, (prisoner_id, bed_number) in enumerate(assignment.items(), start=1):
    # bed_number is 1-indexed, so we subtract 1 to get the DataFrame row position
    bed_row = cell_spaces.iloc[bed_number - 1]
    
    # Get prisoner details from prisoners_encoded using prisoner_id
    prisoner_details = next((p for p in prisoners_encoded if int(p['prisoner_id']) == int(prisoner_id), None)
    
    # Print prisoner details along with assigned bed details in a readable format
    print(f"[{i}] Prisoner {prisoner_details['name']} (ID: {prisoner_id}) → Bed {bed_row['bed_id']}")
    print(f"    Prisoner: Sex={prisoner_details['sex']}, Age={prisoner_details['age_category']}, Supervision={prisoner_details['supervision_level']}")
    
    # Decode bed attributes using the encoder's categories
    bed_sex = encoder.categories_[list(non_numerical_cols).index('sex_assignment')][int(bed_row['sex_assignment'])]
    bed_age = encoder.categories_[list(non_numerical_cols).index('age_category')][int(bed_row['age_category'])]
    bed_supervision = encoder.categories_[list(non_numerical_cols).index('supervision_level')][int(bed_row['supervision_level'])]
    
    print(f"    Bed: Sex={bed_sex}, Age={bed_age}, Supervision={bed_supervision}")
    print()

## Results

The Essence model above defines:
- **Given parameters**: Sets of prisoner and bed IDs, plus attribute functions for sex, age, and supervision level
- **Domains**: Finite integer domains based on the actual prisoner and bed IDs
- **Decision variable**: `assignment` - a total function mapping each prisoner to a bed
- **Constraints**:
  - Injectivity: Each bed assigned to at most one prisoner
  - Sex matching: Prisoners assigned to beds with same sex
  - Age matching: Prisoners assigned to beds with same age category  
  - Supervision matching: Prisoners assigned to beds with same supervision level

Now let's run the model and examine the results.